In [68]:
import requests
import pandas as pd
import os
import numpy as np

def download_and_save_data(url,save_filename):
    r = requests.get(url, allow_redirects=True)#download csv file

    #save string as a csv file
    with open(save_filename, "w") as text_file:
        print(r.text, file=text_file)
        
def add_network_information_to_df(df,net,suffix):
    
    net=net.loc[df['FIPS'],df['FIPS']] # select only counties present in the Corona data
    
    print(net.shape)
    
    net.columns = net.columns.astype('int').astype('str')#make FIPS code a string
    net.columns = suffix + '_' + net.columns.values
    
    net = net.reset_index() # make own_county into a column for dataframe

    print(net.shape)
    
    df=pd.merge(df,net,left_on='FIPS', right_on='own_county', how='inner')
    df=df.drop('own_county',1)
    
    print(df.shape)

    return df

# Update the county-level cases of corona and death rates

In [69]:
save_filename = "county_corona_cases.csv"
url='https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv?_ga=2.56660078.575217947.1584705863-1986476536.1584542614'
download_and_save_data(url,save_filename)

save_filename = "county_corona_deaths.csv"
url = 'https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv?_ga=2.44956710.385475960.1585304312-1986476536.1584542614'
download_and_save_data(url,save_filename)

# Download county geographic adjacency matrix from Census Bureau

In [70]:
url = 'https://www2.census.gov/geo/docs/reference/county_adjacency.txt'
save_filename = "geography_adj_tab.csv"

download_and_save_data(url,save_filename)

# Start building dataframe with recorded COVID-19 cases

In [71]:
corona = pd.read_csv('county_corona_cases.csv')
corona = corona[corona.columns[[0,1,2,-37]]]

#change column name for most recent measure of # cases
cols=corona.columns.values
cols[-1] = 'corona_cases'
cols[0] = 'countyFIPS'
corona.columns = cols

corona = corona[corona['countyFIPS'] != 0] #remove cases not allocated to a county

#incorrect FIPS code corrections
FIPS = corona['countyFIPS']
FIPS[FIPS == 49040] = 49049
FIPS[FIPS == 20710] = 20107
corona['countyFIPS'] = FIPS

#corona = corona[corona['corona_cases'] > 0]
#corona = corona.nlargest(500,'corona_cases')
corona.shape

/Library/Python/3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


(3145, 4)

# add COVID-19 death data

In [72]:
death = pd.read_csv("county_corona_deaths.csv")

death = death[death.columns[[0,1,2,-27]]]

#change column name for most recent measure of # cases
cols=death.columns.values
cols[-1] = 'corona_deaths'
cols[0] = 'countyFIPS'
death.columns = cols

death = death[death['countyFIPS'] != 0] #remove cases not allocated to a county

#incorrect FIPS code corrections
FIPS = death['countyFIPS']
FIPS[FIPS == 49040] = 49049
FIPS[FIPS == 20710] = 20107
death['countyFIPS'] = FIPS

#death = death[death['corona_cases'] > 0]
#corona = corona.nlargest(500,'corona_cases')
corona = pd.merge(corona,death[['countyFIPS','corona_deaths']],left_on='countyFIPS', right_on='countyFIPS',how='inner')
corona.shape

/Library/Python/3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


(3145, 5)

# Download country-level presidential election result data from MIT election lab

In [73]:

politics = pd.read_csv('countypres_2000-2016.csv')

politics = politics[politics['year'] == 2016]
politics = politics[politics['party'].isin(['democrat','republican'])]
politics['voteshare'] = politics['candidatevotes']/politics['totalvotes']
#politics['voteshare_diff'] = politics.groupby(['state','FIPS'],as_index=True)['voteshare'].diff(-1) #* (-1)
#politics = politics[['state','FIPS','voteshare_diff']].dropna()

dem_votes = politics[politics['party'] == 'democrat']
rep_votes = politics[politics['party'] == 'republican']

votes = pd.merge(dem_votes[['FIPS','candidatevotes']],
         rep_votes[['FIPS','candidatevotes']],
         right_on='FIPS',left_on='FIPS')

votes=votes.rename(columns={'candidatevotes_x': 'dem_votes',
                     'candidatevotes_y': 'rep_votes'})


politics = pd.merge(politics,dem_votes,left_on='FIPS',right_on='FIPS')[['FIPS','voteshare_y']]
politics = politics.rename(columns={'voteshare_y':'dem_share'})

politics = pd.merge(politics,rep_votes,left_on='FIPS',right_on='FIPS')[['FIPS','state','dem_share','voteshare']]
politics = politics.rename(columns={'voteshare':'rep_share'})
politics['voteshare_diff'] = politics['dem_share'] - politics['rep_share']
politics['polarization'] = politics['voteshare_diff'].abs()

politics=politics.drop_duplicates()

politics = pd.merge(politics,votes,left_on='FIPS', right_on='FIPS')

#join and select relevent columns
df=pd.merge(corona,politics,left_on='countyFIPS',right_on='FIPS',how='inner')
df = df[['FIPS','state','County Name','voteshare_diff','polarization','dem_share','rep_share','corona_cases','corona_deaths']]
df.shape

(3114, 9)

# Download data for proportion of people without health insurance from US Census Bureau 

In [74]:
health = pd.read_csv('SAHIE_25MAR20_13_22_18_92.csv')[['ID','Uninsured: %','Uninsured: Number']]
health=health.rename(columns = {'ID' : 'FIPS','Uninsured: %' : 'prop_uninsured','Uninsured: Number' : 'num_uninsured'})

import string 

## convert the messed up column to all floats (some are strings with commas)

def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

num_insured = health['num_uninsured']
for i in num_insured.index.values:
    s = num_insured[i]

    if type(s) == str: num_insured[i] = float(remove_punctuation(s))
    else: num_insured[i] = float(s)
        

health['num_uninsured'] = num_insured.astype('float')*100

df=pd.merge(df,health,left_on='FIPS',right_on='FIPS',how='inner')

df.shape

/Library/Python/3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Library/Python/3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


(3114, 11)

# Download data for proportions of obesity for counties from the CDC (2013 data)

In [75]:
obesity = pd.read_csv('Obesity_2013.csv')
obesity = obesity.rename(columns={'region':'FIPS','value':'obesity'})
df = pd.merge(df,obesity,left_on='FIPS',right_on='FIPS',how='inner')
df.shape

(3110, 12)

# Number of ICU beds available in each county from the Kaiser Health News website

In [76]:
ICU = pd.read_excel('KHN_ICU_bed_county_analysis_2.xlsx')
ICU=ICU[['cnty_fips','all_icu']]
ICU.columns= ['FIPS','ICU_beds']
df = pd.merge(df,ICU,left_on='FIPS', right_on='FIPS')
df.shape

(3110, 13)

# Add per capita Facebook connectivity between U.S counties and foreign countries. Using the Facebook SCI (contact Facebook for access)

In [77]:
sci_country = pd.read_csv('County_Country.csv')

sci_country = sci_country[sci_country['friend_country'].isin(['CN','IT','ES','DE','IR','KR','GB'])][['friend_country','own_county','sci_cntry']]
sci_country = sci_country.pivot(index='own_county', columns='friend_country', values='sci_cntry')
sci_country.columns = 'SCI_' + sci_country.columns.values
sci_country = sci_country.reset_index()

df=pd.merge(df,sci_country,left_on='FIPS',right_on='own_county',how='inner')
df=df.drop('own_county',1)

df.shape

(3104, 20)

# Add country-level age data using US Census Bureau data

In [78]:
'''
1 = Age 0 to 4 years
2 = Age 5 to 9 years
3 = Age 10 to 14 years
4 = Age 15 to 19 years
5 = Age 20 to 24 years
6 = Age 25 to 29 years
7 = Age 30 to 34 years
8 = Age 35 to 39 years
9 = Age 40 to 44 years
10 = Age 45 to 49 years
11 = Age 50 to 54 years
12 = Age 55 to 59 years
13 = Age 60 to 64 years
14 = Age 65 to 69 years
15 = Age 70 to 74 years
16 = Age 75 to 79 years
17 = Age 80 to 84 years
18 = Age 85 years or older'''

age=pd.read_csv('cc-est2018-alldata.csv', encoding = "ISO-8859-1") #load data

#create combined FIPS code
age['COUNTY'] = [a.zfill(3) for a in age['COUNTY'].astype('str')]
age['STATE'] = [a.zfill(2) for a in age['STATE'].astype('str')]
age['FIPS'] = (age['STATE'] + age['COUNTY']).astype(float)

#age = age[['AGEGRP','TOT_POP']][age['AGEGRP'] == 1]
age[['AGEGRP']] = age['AGEGRP']*5

#only most recent year
age = age[age['YEAR'] == age['YEAR'].max()]

# create total-population column
TOT_POP = age[age['AGEGRP'] == 0][['FIPS', 'TOT_POP']]
age = age[age['AGEGRP'] != 0][['FIPS', 'AGEGRP', 'TOT_POP']]
age=pd.merge(age,TOT_POP, left_on=['FIPS'], right_on=['FIPS'])

#adjust age to mid point in age categories
age['AGEGRP'] = age['AGEGRP'] - 2.5

#proportion of people in each age category
age['age_proportion'] = age['TOT_POP_x']/age['TOT_POP_y']

#average age in each county
age['mean_age'] = age['AGEGRP']*age['age_proportion']
age=age.groupby(['FIPS'],as_index=False).sum()[['FIPS','mean_age']]

#add age to dataframe
df = pd.merge(df,age,left_on=['FIPS'], right_on=['FIPS'],how='inner')

df.shape

(3104, 21)

# add country-level population data from US Census Bureau

In [79]:
#get 2018 population estimates for counties
population = pd.read_excel('PopulationEstimates.xls',skiprows=range(1),header=1)
population = population[['FIPS', 'POP_ESTIMATE_2018','Rural-urban_Continuum Code_2013']]

#change name of population column
population = population.rename(columns={"POP_ESTIMATE_2018": "population",
                                       'Rural-urban_Continuum Code_2013':'urban_rural'})

#add population to dataframe
df=pd.merge(df,population,left_on='FIPS',right_on='FIPS',how='inner')

df.shape

(3104, 23)

# add country-level population density, income and education data from US Census Bureau

In [80]:
density=pd.read_csv("DEC_10_SF1_GCTPH1.US05PR_with_ann.csv", encoding = "ISO-8859-1",skiprows=1)
density = density[['Target Geo Id2','Density per square mile of land area - Population','Density per square mile of land area - Housing units']]
density.columns = ['FIPS','density_pop','density_house']

df=pd.merge(df,density,left_on='FIPS',right_on='FIPS',how='inner')
df.shape

#add 2018 median incoem data to dataframe
income = pd.read_excel('Unemployment.xls',skiprows=range(1),header=3)
income = income[['FIPS', 'Median_Household_Income_2018']]
income = income.rename(columns={"Median_Household_Income_2018": "income",})

df=pd.merge(df,income,left_on='FIPS',right_on='FIPS',how='inner')
df.shape

#add 2018 median incoem data to dataframe
education = pd.read_excel('Education.xls',skiprows=range(1),header=3)
education = education[['FIPS Code', "Percent of adults with a bachelor's degree or higher, 2014-18"]]
education = education.rename(columns={"FIPS Code":"FIPS", "Percent of adults with a bachelor's degree or higher, 2014-18": "education"})
education['FIPS'] = education['FIPS'].astype('float')

df=pd.merge(df,education,left_on='FIPS',right_on='FIPS',how='inner')
df.shape

(3104, 27)

# add geography adjacency network, US Census Bureau data 

In [81]:
geo_adj = pd.read_csv('geography_adj_tab.csv',sep='\t',header=None)[[1,3]].fillna(method='ffill')
geo_adj.columns = ['own_county','neighbour_county']
geo_adj['weight'] = 1

geo_adj = geo_adj.pivot(index='own_county', columns='neighbour_county', values='weight').fillna(0)

print(geo_adj.shape)

df = add_network_information_to_df(df,geo_adj,suffix='geo')
df.shape

#geo_adj.head(8)

(3234, 3234)
(3104, 3104)
(3104, 3105)
(3104, 3131)


(3104, 3131)

# Add network of Facebook friends between each county pair using Facebook SCI data

In [82]:
#add SCI facebook data to df
net = pd.read_csv('County_County.csv')

# long version to wide version
net = net[['own_county','friend_county','sci']].pivot(index='own_county', columns='friend_county', values='sci')
df = add_network_information_to_df(df,net, suffix='net')
df.shape

#net.head(8)

(3104, 3104)
(3104, 3105)
(3104, 6235)


(3104, 6235)

# save dataframe

In [83]:
#make FIPS code a string
df['FIPS'] = df['FIPS'].astype('int').astype('str')

#save to csv
df.to_csv('county_level_df',index=False)
df.shape

(3104, 6235)

In [84]:
df.shape

(3104, 6235)